In [1]:
import numpy as np
import pandas as pd
import tabula

The following loop is very slow because it has to open and close pdf file many times.
After that I found the multiple_tables option...
It is much faster than the stupid loop, though the form of the dataframes in the list are different.
That is, the cleaning process here is not compatible with the argument...
Maybe try it next time...

In [2]:
#tabula.read_pdf('data/FinInstList.pdf', pages='all', multiple_tables=True)

dfs = [tabula.read_pdf('data/FinInstList.pdf', pages=str(i+1)) for i in range(287)]

Check the dataframes in the list

First Dataframe

How many dataframes with exactly 5 columns

Where those mutant dataframes are, and how they look like

In [3]:
dfs[0].head()

,序号,中文全称,英文全称,金融许可证机构编码,机构类型
0,1.0,国家开发银行,China Development Bank,G0001H111000001,开发性金融机构
1,2.0,中国进出口银行,The Export-Import Bank of China,A0003H111000001,政策性银行
2,3.0,中国农业发展银行,Agricultural Development Bank of China,A0002H111000001,政策性银行
3,NaN,NaN,INDUSTRIAL AND COMMERCIAL BANK OF CHINA,NaN,NaN
4,4.0,中国工商银行股份有限公司,NaN,B0001H111000001,国有大型商业银行


In [4]:
lengths = np.array([len(df.columns) for df in dfs])
np.count_nonzero(lengths==5)

282

In [5]:
np.argwhere(lengths != 5)

array([[177],
       [202],
       [203],
       [204],
       [205]])

In [6]:
dfs[177].head()

,Unnamed: 0,Unnamed: 1,Guangdong GaoyaoRural Commercial,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,2829.0,广东高要农村商业银行股份有限公司,NaN,NaN,B1377H344120001,农村商业银行
1,NaN,NaN,Bank Company Limited,NaN,NaN,NaN
2,NaN,NaN,Guangdong Huaiji Rural Commercial Bank,NaN,NaN,NaN
3,2830.0,广东怀集农村商业银行股份有限公司,NaN,NaN,B1785H344120001,农村商业银行
4,NaN,NaN,Company Limited,NaN,NaN,NaN


In [7]:
dfs[202].head()

,3229,贵州独山农村商业银行股份有限公司,B0391H252270001,农村商业银行
0,3230,贵州花溪农村商业银行股份有限公司,B0419H252010001,农村商业银行
1,3231,贵州安龙农村商业银行股份有限公司,B0438H252230001,农村商业银行
2,3232,贵州贞丰农村商业银行股份有限公司,B0439H252230001,农村商业银行
3,3233,贵州紫云农村商业银行股份有限公司,B0464H252040001,农村商业银行
4,3234,贵州都匀农村商业银行股份有限公司,B0507H252270001,农村商业银行


Clean 0 and 177 dataframes first,

Put the information in column labels to the table

Concatenate!

In [8]:
dfs[0].rename(columns={dfs[0].columns[-2]: 'instcode', dfs[0].columns[-1]: 'type'}, inplace=True)
dfs[177].rename(columns={dfs[177].columns[-2]: 'instcode', dfs[177].columns[-1]: 'type'}, inplace=True)

In [9]:
def clean_small_df(df):
    small = pd.DataFrame(np.array(df.columns)[np.newaxis, :], columns=df.columns)
    
    large = pd.concat([df, small], ignore_index=True).iloc[:, -2:]
    large.columns = ['instcode', 'type']
    
    return large

In [10]:
dfs_cleaned = [clean_small_df(dfs[i]) if i not in (0, 177) else dfs[i].iloc[:, -2:] for i in range(287)]

In [11]:
df_main = pd.concat(dfs_cleaned, ignore_index=True)

Drop all the duplicates as there should not be any duplicate in the clean data!

In [12]:
df_main = df_main.dropna().drop_duplicates(keep=False)

len(df_main)

4589

In [13]:
df_main.head()

,instcode,type
0,G0001H111000001,开发性金融机构
1,A0003H111000001,政策性银行
2,A0002H111000001,政策性银行
4,B0001H111000001,国有大型商业银行
6,B0002H111000001,国有大型商业银行


In [14]:
df_main.to_excel('data/FinInstList.xlsx')